In [ ]:
def request(url, params = None, tag = None, attrs = None):
    import requests as req
    try:
        from BeautifulSoup import BeautifulSoup
    except ImportError:
        from bs4 import BeautifulSoup

    headers = {
        'User-Agent': 'test#1234', #Enter your discord handle
        'From': 'email@gmail.com' #Enter your gmail
        }

    response = req.get(url, headers = headers, params = params).text

    return response

def remove_past_price(dir):

    print('------------Remove past price starting.------------')

    #90 Days is the history limit. The current GEWatch uses 7 days of data, so you may edit this.
    dt_90d = datetime.datetime.now().replace(minute=0, second=0, microsecond=0) - datetime.timedelta(days = 90)

    try:
      with open(dir, "r") as file:
          bulk_prices_history = json.load(file)
          file.close()
    except:
        print('Price history not found.')
        return

    past_prices_history = {key:bulk_prices_history[key] for key in bulk_prices_history if datetime.datetime.strptime(key, '%Y-%m-%d %H:%M:%S') < dt_90d}

    if len(past_prices_history.keys()) > 0:
        print('Earliest found invalid date: ', min(past_prices_history.keys()))
        print('Latest found invalid date: ', max(past_prices_history.keys()))
    elif len(bulk_prices_history.keys()) > 0:
        print('Past price history not found.')
        return bulk_prices_history
    else:
        print('Price history not found.')
        return {}

    bulk_prices_history = {key:bulk_prices_history[key] for key in bulk_prices_history if datetime.datetime.strptime(key, '%Y-%m-%d %H:%M:%S') >= dt_90d}

    print('History cleaned, ', min(bulk_prices_history.keys()), ' to ', max(bulk_prices_history.keys()), ' is left.')

    return bulk_prices_history

def update_to_latest_price(dir, bulk_prices_history, last_time = None):

    print('------------Update to latest price starting.------------')

    i = 1
    while(True):

        dt = datetime.datetime.now().replace(minute=0, second=0, microsecond=0) - datetime.timedelta(hours = 1)
        print('Update to latest price iteration: ', i)
        #print('Current Time: ', dt)
        #print('Timestamp: ', datetime.datetime.timestamp(dt))
        #print('Timestamp UTC: ', datetime.datetime.timestamp(dt - datetime.timedelta(hours = 3)))

        if last_time == None:
            last_time = max(bulk_prices_history.keys(), default = (datetime.datetime.now().replace(minute=0, second=0, microsecond=0) - datetime.timedelta(days = 90)).strftime('%Y-%m-%d %H:%M:%S'))
        else:
            last_time = (datetime.datetime.strptime(last_time, '%Y-%m-%d %H:%M:%S') + datetime.timedelta(hours = 1)).strftime('%Y-%m-%d %H:%M:%S')

        if datetime.datetime.timestamp(dt) > datetime.datetime.timestamp(datetime.datetime.strptime(last_time, '%Y-%m-%d %H:%M:%S')):
            h_list = json.loads(request('https://prices.runescape.wiki/api/v1/osrs/1h', params = {'timestamp': int(datetime.datetime.timestamp(datetime.datetime.strptime(last_time, '%Y-%m-%d %H:%M:%S') + datetime.timedelta(hours=1)))}))
            h_list['timestamp'] = datetime.datetime.fromtimestamp(h_list['timestamp']).strftime('%Y-%m-%d %H:%M:%S')
            bulk_prices_history[h_list['timestamp']] = h_list
        else:
            print ('Data is up to date.')
            break

        print('Loaded: ', h_list['timestamp'])

        if i%24 == 0:
            print('Starting to save, do not close.')
            now = datetime.datetime.timestamp(datetime.datetime.now())
            try:
                with open(dir, "w") as file:
                    json.dump(bulk_prices_history, file)
                    file.close()
            except:
                with open(dir, "w") as file:
                    file.write('')
                    json.dump(bulk_prices_history, file)
                    file.close()
            print('Finished saving. Took ', int(datetime.datetime.timestamp(datetime.datetime.now()) - now) ,' seconds.')

        i += 1
        time.sleep(10)


    if i != 1:
        print('Starting the final save, do not close.')
        now = datetime.datetime.timestamp(datetime.datetime.now())
        try:
            with open(dir, "w") as file:
                json.dump(bulk_prices_history, file)
                file.close()
        except:
            with open(dir, "w") as file:
                file.write('')
                json.dump(bulk_prices_history, file)
                file.close()
        print('Finished saving. Took ', int(datetime.datetime.timestamp(datetime.datetime.now()) - now) ,' seconds.')

def update_price(dir):

    print('------------Update price starting.------------')

    i = 1
    while(True):

        dt = datetime.datetime.now().replace(minute=0, second=0, microsecond=0)
        #print('Current Time: ', dt)
        #print('Timestamp: ', datetime.datetime.timestamp(dt))
        #print('Timestamp UTC: ', datetime.datetime.timestamp(dt - datetime.timedelta(hours = 3)))
        with open(dir, "r") as file:
            bulk_prices_history = json.load(file)
            file.close()


        future = dt + datetime.timedelta(hours = 1)
        if (dt - datetime.timedelta(hours = 1)).strftime('%Y-%m-%d %H:%M:%S') not in bulk_prices_history:
            print('Update price iteration: ', i)
            h_list = json.loads(request('https://prices.runescape.wiki/api/v1/osrs/1h', params = {'timestamp': int(datetime.datetime.timestamp(dt - datetime.timedelta(hours = 1)))}))
            h_list['timestamp'] = datetime.datetime.fromtimestamp(h_list['timestamp']).strftime('%Y-%m-%d %H:%M:%S')
            bulk_prices_history[h_list['timestamp']] = h_list

            print('Loaded: ', h_list['timestamp'])
            print('Dt: ', dt)

            i += 1

            print('Starting save, do not close.')
            now = datetime.datetime.timestamp(datetime.datetime.now())
            try:
                with open(dir, "w") as file:
                    json.dump(bulk_prices_history, file)
                    file.close()
            except:
                with open(dir, "w") as file:
                    file.write('')
                    json.dump(bulk_prices_history, file)
                    file.close()
            print('Finished saving. Took ', int(datetime.datetime.timestamp(datetime.datetime.now()) - now) ,' seconds.')

        animation = "_\\|/_"
        idx = 0
        while dt + datetime.timedelta(hours = 1) > datetime.datetime.now():
            print('Waiting the next hour ', animation[idx % len(animation)], end="\r")
            idx += 1
            time.sleep(0.2)

        #time.sleep((future - dt).total_seconds() + 300)
        time.sleep(300)

import json
import time
import os
import datetime

dir_path = '/content/drive/MyDrive/Colab/GEWatch/content' #Price history file location. This is set for drive, you may want to change it.
data_file = 'Price_History.json' #Try to save back ups from time to time. If the connection cuts off midsave file becomes corrupted.
data_dir = os.path.join(dir_path, 'Data', data_file)

if not os.path.exists(os.path.join(dir_path, 'Data')):
    os.makedirs(os.path.join(dir_path, 'Data'))

#latest_flag = input('[Y] to update to latest.')

update_to_latest_price(data_dir, remove_past_price(data_dir))
#if latest_flag == 'Y': update_to_latest_price(data_dir)
update_price(data_dir)

------------Remove past price starting.------------
Earliest found invalid date:  2023-07-24 06:00:00
Latest found invalid date:  2023-07-24 15:00:00
History cleaned,  2023-07-24 16:00:00  to  2023-10-22 12:00:00  is left.
------------Update to latest price starting.------------
Update to latest price iteration:  1
Loaded:  2023-10-22 13:00:00
Update to latest price iteration:  2
Loaded:  2023-10-22 14:00:00
Update to latest price iteration:  3
Loaded:  2023-10-22 15:00:00
Update to latest price iteration:  4
Data is up to date.
Starting the final save, do not close.
Finished saving. Took  65  seconds.
------------Update price starting.------------
Update price iteration:  1
Loaded:  2023-10-22 16:00:00
Dt:  2023-10-22 17:00:00
Starting save, do not close.
Finished saving. Took  70  seconds.
Update price iteration:  2
Loaded:  2023-10-22 17:00:00
Dt:  2023-10-22 18:00:00
Starting save, do not close.
Finished saving. Took  62  seconds.
Update price iteration:  3
Loaded:  2023-10-22 18:0

# END
